**DISCLAIMER**

By accessing this code, you acknowledge the code is made available for presentation and demonstration purposes only and that the code (1) is not subject to SOC 1 and SOC 2 compliance audits, and (2) is not designed or intended to be a substitute for the professional advice, diagnosis, treatment, or judgment of a certified financial services professional. Do not use this code to replace, substitute, or provide professional financial advice, or judgement. You are solely responsible for ensuring the regulatory, legal, and/or contractual compliance of any use of the code, including obtaining any authorizations or consents, and any solution you choose to build that incorporates this code in whole or in part.

In [ ]:
%sh ls /dbfs/FileStore/geospatial_fraud_detection/synapse_migration


anomalies_miami_b8fe8ae7-3450-416f-ae75-a03fb0488fbf.csv
miami.csv
personalized_miami_data_b07db0f6-7958-4f8e-b2ea-b80e775ff48b.csv

In [ ]:
%pip install geojson

Python interpreter will be restarted.
Collecting geojson
 Using cached geojson-2.5.0-py2.py3-none-any.whl (14 kB)
Installing collected packages: geojson
Successfully installed geojson-2.5.0
Python interpreter will be restarted.

In [ ]:
import pandas as pd
import numpy
import matplotlib.pyplot as plt
import geojson
import folium
from folium import plugins
import requests

In [ ]:
root_dir = "/dbfs/FileStore/geospatial_fraud_detection/synapse_migration"
miami_df = pd.read_csv(f'{root_dir}/miami.csv')
anomalies_miami = pd.read_csv(f'{root_dir}/anomalies_miami_b8fe8ae7-3450-416f-ae75-a03fb0488fbf.csv')
personalized_miami = pd.read_csv(f'{root_dir}/personalized_miami_data_b07db0f6-7958-4f8e-b2ea-b80e775ff48b.csv')
santorini_anomalies = pd.read_csv(f'{root_dir}/santorini_anomalies.csv')

In [ ]:
santorini_anomalies = spark.createDataFrame(santorini_anomalies)

In [ ]:
santorini_anomalies.write.saveAsTable('geospatial_miami.SuspiciousTransactionsSantorini')

In [ ]:
%scala
//Configuration settings to connect to blob storage
spark.conf.set("fs.azure.account.key.#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net","#STORAGE_ACCOUNT_KEY#")

//declaring the credentials
//val sqluser =  dbutils.secrets.get(scope="esgprod", key="sqluser")
//val sqlpassword = dbutils.secrets.get(scope="esgprod", key="sqlpassword")
val dbtable = "ADB_SuspiciousTransactionsSantorini" //Synapse Table Name Set here
val url = s"jdbc:sqlserver://#WORKSPACE_NAME#.sql.azuresynapse.net:1433;database=#DATABASE_NAME#;user=#SQL_USERNAME#;password=#SQL_PASSWORD#;encrypt=true;trustServerCertificate=true;hostNameInCertificate=*.sql.azuresynapse.net;loginTimeout=30;authentication=ActiveDirectoryPassword"

//Read only newer records from our intermediary table in Databricks
var migrate_df = spark.sql("SELECT latitude, longitude, amount, current_timestamp() as ReportedOn FROM geospatial_miami.SuspiciousTransactionsSantorini")
 
print(s"Total Records", migrate_df.count())
print("\n")

//Send new records to synapse for further processing
migrate_df.write.format("com.databricks.spark.sqldw").option("forwardSparkAzureStorageCredentials", "true").mode("append").option("url", url).option("dbtable", dbtable).option("tempDir", "abfss://esg-migrate@#STORAGE_ACCOUNT_NAME#.dfs.core.windows.net/tempDirs").save()

(Total Records,4)
sqluser: String = [REDACTED]
sqlpassword: String = [REDACTED]
dbtable: String = ADB_SuspiciousTransactionsSantorini
url: String = jdbc:sqlserver://marketdatasynapse.sql.azuresynapse.net:1433;database=FSIRiskDW;user=[REDACTED];password=[REDACTED];encrypt=true;trustServerCertificate=true;hostNameInCertificate=*.sql.azuresynapse.net;loginTimeout=30;authentication=ActiveDirectoryPassword
migrate_df: org.apache.spark.sql.DataFrame = [latitude: double, longitude: double ... 2 more fields]